# Django-Geo-SPaaS - GeoDjango framework for Satellite Data Management


#### First of all we need to initialize Django to work. Let's do some 'magic'

In [1]:
import os, sys
os.environ['DJANGO_SETTINGS_MODULE'] = 'geospaas_project.settings'
sys.path.insert(0, '/vagrant/shared/course_vm/geospaas_project/')

import django
django.setup()

from django.conf import settings

#### Now we can import our models

In [2]:
from geospaas.catalog.models import Dataset
from geospaas.catalog.models import DatasetURI

#### Now we can use the model Dataset to search for datasets

In [3]:
# find all images
datasets = Dataset.objects.all()

### What is happening:
 - A SQL query is generated
 - The query is sent to the database (local database driven by SpatiaLite)
 - The query is executed by the database engine
 - The result is sent back to Python
 - The results is wrapped into a QuerySet object

In [4]:
print datasets.count()

# print info about each image
for ds in datasets:
    print ds

5
ENVISAT/MERIS/2012-03-03T09:38:10.423969+00:00
ENVISAT/ASAR/2008-11-10T20:56:23.207430+00:00
AQUA/MODIS/2015-05-01T11:36:49+00:00
AQUA/MODIS/2015-05-02T12:20:09+00:00
SENTINEL-1A/SAR/2015-07-02T17:29:54.066281+00:00


#### Use the complex structure of the catalog models

In [12]:
# get just one Dataset
ds0 = datasets.first()
print ds0.time_coverage_start


2012-03-03 09:38:10.423969+00:00


datetime.datetime(2008, 11, 10, 20, 56, 23, 207430, tzinfo=<UTC>)

In [6]:
# print joined fields (Foreign key)
for ds in datasets:
    print ds.source.instrument.short_name,
    print ds.source.platform.short_name

    
    

MERIS ENVISAT
ASAR ENVISAT
MODIS AQUA
MODIS AQUA
SAR SENTINEL-1A


In [7]:
# get infromation from Foreign key in the opposite direction
print ds0.dataseturi_set.first().uri


file://localhost/vagrant/shared/test_data/meris_l1/MER_FRS_1PNPDK20120303_093810_000000333112_00180_52349_3561.N1


## Search for data

In [15]:
# search by time
ds = Dataset.objects.filter(time_coverage_start='2012-03-03 09:38:10.423969')
print ds

ds = Dataset.objects.filter(time_coverage_start__gte='1900-03-01')
print ds

[<Dataset: ENVISAT/MERIS/2012-03-03T09:38:10.423969+00:00>]
[<Dataset: ENVISAT/MERIS/2012-03-03T09:38:10.423969+00:00>, <Dataset: ENVISAT/ASAR/2008-11-10T20:56:23.207430+00:00>, <Dataset: AQUA/MODIS/2015-05-01T11:36:49+00:00>, <Dataset: AQUA/MODIS/2015-05-02T12:20:09+00:00>, <Dataset: SENTINEL-1A/SAR/2015-07-02T17:29:54.066281+00:00>]


In [16]:
# search by instrument
ds = Dataset.objects.filter(source__instrument__short_name='MODIS')
print ds

[<Dataset: AQUA/MODIS/2015-05-01T11:36:49+00:00>, <Dataset: AQUA/MODIS/2015-05-02T12:20:09+00:00>]


In [25]:
# search by spatial location
ds0 = Dataset.objects.first()
ds0_geom = ds0.geographic_location.geometry

ds_ovlp = Dataset.objects.filter(
    geographic_location__geometry__intersects=ds0_geom,
    time_coverage_start__gte='2015-05-02',
    source__platform__short_name='AQUA')
print ds_ovlp


[<Dataset: AQUA/MODIS/2015-05-02T12:20:09+00:00>]


### Finally, get data

In [26]:
dsovlp0 = ds_ovlp.first()

In [31]:
uri0 = dsovlp0.dataseturi_set.first().uri
print uri0

file://localhost/vagrant/shared/test_data/obpg_l2/A2015122122000.L2_LAC.NorthNorwegianSeas.hdf


In [33]:
from nansat import Nansat
n = Nansat(uri0.replace('file://localhost', ''))
print n[1]

















[[ 22.20515633  22.15754128  22.10977554 ...,  -2.4479444   -2.48379612
   -2.51980472]
 [ 22.20428276  22.15644073  22.10901642 ...,  -2.45618773  -2.49206471
   -2.52809858]
 [ 22.2034626   22.15546989  22.10785866 ...,  -2.46444297  -2.50034523
   -2.53640437]
 ..., 
 [ 19.98286629  19.89513779  19.80788994 ..., -20.16187286 -20.20990181
  -20.25743484]
 [ 19.98246765  19.8946743   19.8073597  ..., -20.18548775 -20.23342133
  -20.28373528]
 [ 19.98209     19.89422989  19.80684853 ..., -20.21025085 -20.25760651
  -20.30633354]]
